In [1]:
import pandas as pd
import numpy as np

In [2]:
pre = pd.read_csv('1merged.csv')

In [3]:
post = pd.read_csv('2merged.csv')

In [4]:
di = pd.read_csv('dateianuarie.csv')

In [5]:
pre.shape

(160, 20)

In [6]:
post.shape

(156, 20)

In [7]:
pre.columns = ['code', 'level', 'load', 'time',
       'rrate', 'hrate', 'ventilation',
       'vo2', 'vo2rel', 'vco2',
       'vco2rel', 'rer', 'vo2eq',
       'vo2pulse', 'fat', 'carb', 'fat100',
       'carb100', 'energy', 'met']

In [8]:
post.columns = ['code', 'level', 'load', 'time',
       'rrate', 'hrate', 'ventilation',
       'vo2', 'vo2rel', 'vco2',
       'vco2rel', 'rer', 'vo2eq',
       'vo2pulse', 'fat', 'carb', 'fat100',
       'carb100', 'energy', 'met']

In [9]:
pre.insert(1, 'name', 'n')

In [10]:
post.insert(1, 'name', 'n')

In [11]:
for i in pre.index:
    pre.loc[i, 'name'] = pre.loc[i-i%4, 'load']

In [12]:
for i in post.index:
    post.loc[i, 'name'] = post.loc[i-i%4, 'load']

In [13]:
pre = pre[['name', 'level', 'rrate', 'hrate',
       'ventilation', 'vo2', 'vo2rel', 'vco2', 'vco2rel', 'rer', 'vo2eq',
       'vo2pulse', 'fat', 'carb', 'fat100', 'carb100', 'energy', 'met']]

In [14]:
post = post[['name', 'level', 'rrate', 'hrate',
       'ventilation', 'vo2', 'vo2rel', 'vco2', 'vco2rel', 'rer', 'vo2eq',
       'vo2pulse', 'fat', 'carb', 'fat100', 'carb100', 'energy', 'met']]

In [15]:
pre = pre.loc[pre.level!='Nume']

In [16]:
post = post.loc[post.level!='Nume']

In [17]:
pre.name = pre.name.str.replace('*','')

In [18]:
pre.loc[pre.name.str.endswith('*')].name.unique()

array([], dtype=object)

In [20]:
pre = pre.reset_index(drop=True)

In [21]:
post = post.reset_index(drop=True)

In [22]:
(pre.name == post.name).value_counts()

name
True    117
Name: count, dtype: int64

In [25]:
post.level.unique()

array(['1', '2', '3'], dtype=object)

In [26]:
pre.level.unique()

array(['1', '2', '3'], dtype=object)

In [28]:
pre.level = pre.level.astype('int')
post.level = post.level.astype('int')

In [31]:
pre.insert(1,'time','initial')
post.insert(1,'time','final')

In [32]:
pre.insert(0,'aeroscan_index', [int(1+(i - i%3)/3) for i in range(117)])

In [33]:
post.insert(0,'aeroscan_index', [int(1+(i - i%3)/3) for i in range(117)])

In [34]:
prepost = pd.concat([pre,post], ignore_index=True)

In [35]:
prepost.columns

Index(['aeroscan_index', 'name', 'time', 'level', 'rrate', 'hrate',
       'ventilation', 'vo2', 'vo2rel', 'vco2', 'vco2rel', 'rer', 'vo2eq',
       'vo2pulse', 'fat', 'carb', 'fat100', 'carb100', 'energy', 'met'],
      dtype='object')

In [36]:
prepost_melt = prepost.melt(id_vars=['aeroscan_index', 'name', 'time', 'level'],
             value_vars=['rrate', 'hrate',
       'ventilation', 'vo2', 'vo2rel', 'vco2', 'vco2rel', 'rer', 'vo2eq',
       'vo2pulse', 'fat', 'carb', 'fat100', 'carb100', 'energy', 'met'],
             var_name='param', value_name='value')

In [37]:
prepost_melt.sort_values(by=['aeroscan_index', 'time', 'level'],
                         ascending=[True, False, True], ignore_index=True, inplace=True)

In [40]:
prepost_melt.duplicated(subset=['aeroscan_index', 'name', 'param', 'time', 'level'], keep=False).value_counts()

False    3744
Name: count, dtype: int64

In [46]:
prepost_pivot = prepost_melt.pivot(columns=['param', 'level', 'time'], index=['aeroscan_index', 'name'], values='value')

In [74]:
prepost_melt.param.unique()

array(['rrate', 'hrate', 'ventilation', 'vo2', 'vo2rel', 'vco2',
       'vco2rel', 'rer', 'vo2eq', 'vo2pulse', 'fat', 'carb', 'fat100',
       'carb100', 'energy', 'met'], dtype=object)

# col_order = {lb : i for i, lb in enumerate(prepost_melt.param.unique())}

In [73]:
col_order = {lb : i for i, lb in enumerate(prepost_melt.param.unique())}

In [97]:
prepost_pivot.sort_index(axis=1, level='param', key=lambda x: x.map(col_order), inplace=True)

In [117]:
prepost_pivot.insert(0,'ID','a')

In [132]:
prepost_pivot.ID = pd.read_csv('key.csv').ID.values

In [134]:
prepost_pivot.ID = prepost_pivot.ID.str.replace('a','')

In [138]:
prepost_pivot.ID = prepost_pivot.ID.astype('int')

In [142]:
prepost_pivot.sort_values(by='ID', ignore_index=False, inplace=True)

In [154]:
d = di.loc[(di.ID!='t29') & (di.Group!='Control'), 'Initials':'Sex']

In [203]:
prepost_pivot = prepost_pivot_rc.copy()

In [205]:
prepost_pivot = prepost_pivot.reset_index()

In [208]:
prepost_pivot.insert(2, 'fname', d.Name.values)

In [210]:
prepost_pivot.insert(3, 'fID', d.ID.values)

In [212]:
prepost_pivot.insert(5, 'location', d['Group (school)'].values)

In [218]:
prepost_pivot.insert(6, 'group', d['Group'].values)

In [220]:
prepost_pivot.insert(7, 'sex', d['Sex'].values)

In [222]:
prepost_pivot = prepost_pivot.drop(columns=['aeroscan_index', 'name', 'fname', 'ID'])

C:\Users\THW\AppData\Local\Temp\ipykernel_14608\3940841191.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  prepost_pivot = prepost_pivot.drop(columns=['aeroscan_index', 'name', 'fname', 'ID'])


In [242]:
prepost_pivot_T = prepost_pivot.T.reset_index().copy()

In [244]:
prepost_pivot_T.insert(0,'c','c')

In [251]:
prepost_pivot_T.c = prepost_pivot_T.param + '_l' + prepost_pivot_T.level.astype('str') + '_' + prepost_pivot_T.time.str[0]

In [255]:
prepost_pivot_T = prepost_pivot_T.drop(columns=['param', 'level', 'time'])

In [262]:
prepost_pivot_T.loc[0:3, 'c'] = ['ID', 'location', 'group', 'sex']

In [265]:
a_coded = prepost_pivot_T.T

In [267]:
a_coded.columns = a_coded.loc['c']

In [269]:
a_coded = a_coded.drop(index='c')

In [303]:
a_coded.columns[4:]

Index(['rrate_l1_f', 'rrate_l1_i', 'rrate_l2_f', 'rrate_l2_i', 'rrate_l3_f',
       'rrate_l3_i', 'hrate_l1_f', 'hrate_l1_i', 'hrate_l2_f', 'hrate_l2_i',
       'hrate_l3_f', 'hrate_l3_i', 'ventilation_l1_f', 'ventilation_l1_i',
       'ventilation_l2_f', 'ventilation_l2_i', 'ventilation_l3_f',
       'ventilation_l3_i', 'vo2_l1_f', 'vo2_l1_i', 'vo2_l2_f', 'vo2_l2_i',
       'vo2_l3_f', 'vo2_l3_i', 'vo2rel_l1_f', 'vo2rel_l1_i', 'vo2rel_l2_f',
       'vo2rel_l2_i', 'vo2rel_l3_f', 'vo2rel_l3_i', 'vco2_l1_f', 'vco2_l1_i',
       'vco2_l2_f', 'vco2_l2_i', 'vco2_l3_f', 'vco2_l3_i', 'vco2rel_l1_f',
       'vco2rel_l1_i', 'vco2rel_l2_f', 'vco2rel_l2_i', 'vco2rel_l3_f',
       'vco2rel_l3_i', 'rer_l1_f', 'rer_l1_i', 'rer_l2_f', 'rer_l2_i',
       'rer_l3_f', 'rer_l3_i', 'vo2eq_l1_f', 'vo2eq_l1_i', 'vo2eq_l2_f',
       'vo2eq_l2_i', 'vo2eq_l3_f', 'vo2eq_l3_i', 'vo2pulse_l1_f',
       'vo2pulse_l1_i', 'vo2pulse_l2_f', 'vo2pulse_l2_i', 'vo2pulse_l3_f',
       'vo2pulse_l3_i', 'fat_l1_f', 'fat_l1_

In [320]:
a_melt = a_coded.melt(id_vars=['ID', 'sex', 'location'],
                      value_vars=a_coded.columns[4:].to_list(),
                      var_name='parameter',
                      value_name='value', ignore_index=True)

In [322]:
a_melt.insert(3,'time','t')

In [323]:
a_melt.insert(4,'level','l')

In [327]:
a_melt.time = a_melt.parameter.str[-1]

In [330]:
a_melt.level = a_melt.parameter.str[-3]

In [334]:
a_melt.parameter = a_melt.parameter.str[:-5]

In [338]:
a_melt

,ID,sex,location,time,level,parameter,value
0,t1,f,Jurilovca,f,1,rrate,20.690417
1,t2,m,Baia,f,1,rrate,38.483108
2,t3,f,Jurilovca,f,1,rrate,34.786918
3,t4,f,Baia,f,1,rrate,21.148
4,t5,m,Jurilovca,f,1,rrate,29.927656
...,...,...,...,...,...,...,...
3739,t36,m,Baia,i,3,met,4.550712
3740,t37,m,Jurilovca,i,3,met,3.217695
3741,t38,m,Jurilovca,i,3,met,6.790791
3742,t39,m,Baia,i,3,met,7.371199


In [346]:
a_pivot = a_melt.pivot(columns='parameter',
                       index=['ID', 'sex', 'location', 'level', 'time'],
                       values='value').reset_index()

In [355]:
a_pivot.columns = ['ID', 'sex', 'location', 'level', 'time', 'rrate',
                   'hrate', 'ventilation', 'vo2', 'vo2rel', 'vco2', 
                   'vco2rel', 'rer', 'vo2eq', 'vo2pulse', 'fat', 'carb',
                   'fat100', 'carb100', 'energy', 'met']

In [361]:
a_pivot

,ID,sex,location,level,time,rrate,hrate,ventilation,vo2,vo2rel,...,vco2rel,rer,vo2eq,vo2pulse,fat,carb,fat100,carb100,energy,met
0,t1,f,Jurilovca,1,f,312.105589,100.0,312.105589,0.0,0.0,...,4.193055,1.111431,20.690417,1.141771,16.311019,30.818651,1.027298,29.999704,7.215153,14.675693
1,t1,f,Jurilovca,1,i,295.490776,100.0,295.490776,0.0,0.0,...,3.96984,1.039256,32.180108,1.010792,14.439885,27.053411,0.972611,27.815252,6.049532,13.894438
2,t1,f,Jurilovca,2,f,310.296048,100.0,310.296048,0.0,0.0,...,4.168744,1.167976,19.582856,1.192903,17.041474,30.663501,1.021342,30.022744,7.075416,14.590605
3,t1,f,Jurilovca,2,i,272.761093,100.0,272.761093,0.0,0.0,...,3.664472,1.061712,33.828897,0.9532,13.617147,25.654067,0.897796,28.574504,5.349479,12.825653
4,t1,f,Jurilovca,3,f,325.401693,100.0,325.401693,0.0,0.0,...,4.371685,1.203862,20.252536,1.289412,18.420167,32.974861,1.071063,30.787048,6.822564,15.300896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,t9,m,Baia,1,i,113.725026,43.363773,262.25814,148.533114,56.636227,...,3.519613,0.830091,32.831139,0.746469,10.2256,21.064423,0.899261,23.424147,6.533456,12.318645
230,t9,m,Baia,2,f,457.241019,100.0,457.241019,0.0,0.0,...,5.890462,1.009178,48.252646,1.518826,20.805837,41.27395,1.505013,27.424312,10.471656,20.616618
231,t9,m,Baia,2,i,180.781293,55.432324,326.129741,145.348447,44.567676,...,4.338205,0.866297,28.080595,0.960213,13.15361,24.315601,1.108411,21.937341,7.517315,15.183719
232,t9,m,Baia,3,f,492.611619,100.0,492.611619,0.0,0.0,...,6.346128,1.025595,47.998615,1.662937,22.779955,43.73057,1.621436,26.970276,10.170433,22.211449


In [362]:
a_pivot.columns

Index(['ID', 'sex', 'location', 'level', 'time', 'rrate', 'hrate',
       'ventilation', 'vo2', 'vo2rel', 'vco2', 'vco2rel', 'rer', 'vo2eq',
       'vo2pulse', 'fat', 'carb', 'fat100', 'carb100', 'energy', 'met'],
      dtype='object')